In [1]:
import numpy as np
import pymannkendall as mk
from statsmodels.tsa.stattools import adfuller

from simple.hurst import HurstRS, HurstDMA, HurstDSOD
from simple.funcs import hurst
from simple.jurik import JCFB
from simple.utils import prun, rolling_apply
from simple.chart import chartFigure, updateFigure

In [2]:
period = 512
noise = 0.48

In [3]:
A = ((np.random.rand(period*5)-noise)).cumsum()
B = np.random.rand(period*5) + A[-1]
T = np.hstack((A, B))
len(T)

5120

In [4]:
fig = chartFigure(Price=T, rows=5, height=700,
    HurstDSOD={'row': 2}, hurstSeries={'row': 2}, HurstRS={'row': 2, 'opacity': 0.5}, HurstDMA={'row': 2}, 
    MannKendall={'row': 3}, pre_whitening={'row': 3},
    CFB={'row': 4},
    ADF={'row': 5},
    layout=dict(legend={'y': 1.05})
)

fig.add_vline(x=period*5, line_dash='dot')
for shape in fig.layout.shapes:
    shape["yref"] = "paper"

fig

FigureWidgetResampler({
    'data': [{'name': '<b style="color:sandybrown">[R]</b> Price <i style="color:#fc9944">~5</i>',
              'type': 'scattergl',
              'uid': '346602ae-0bb3-4ce0-b9a1-087450a2a496',
              'x': array([   0,    4,    7, ..., 5109, 5112, 5119], shape=(1000,)),
              'xaxis': 'x5',
              'y': array([ 0.10882556, -0.16986822,  0.65426962, ..., 63.40516378, 64.27397466,
                          64.31954872], shape=(1000,))},
             {'name': 'HurstDSOD',
              'type': 'scattergl',
              'uid': 'bb002ba4-ce56-4700-9b5f-a917e848ed36',
              'x': [],
              'xaxis': 'x5',
              'y': [],
              'yaxis': 'y3'},
             {'name': 'hurstSeries',
              'type': 'scattergl',
              'uid': '5963c78f-2602-4a01-9cba-58731a0ac35b',
              'x': [],
              'xaxis': 'x5',
              'y': [],
              'yaxis': 'y3'},
             {'name': 'HurstRS',
        

In [5]:
h = JCFB(T, 9, 36)
updateFigure(fig, CFB=h)

In [6]:
def calc(X: np.array, Period: int) -> np.array:
    x = rolling_apply(lambda x: mk.original_test(x).Tau, Period, X, n_jobs=128, progress=True)
    x[:Period] = x[Period]
    return x

h = calc(T, period)
updateFigure(fig, MannKendall=h)

  0%|          | 0/4608 [00:00<?, ?it/s]

In [7]:
def calc(X: np.array, Period: int) -> np.array:
    x = rolling_apply(lambda x: mk.trend_free_pre_whitening_modification_test(x).Tau, Period, X, n_jobs=128, progress=True)
    x[:Period] = x[Period]
    return x

h = calc(T, period)
updateFigure(fig, pre_whitening=h)

  0%|          | 0/4608 [00:00<?, ?it/s]

In [8]:
def ADFSeries(X: np.array, Period: int) -> np.array:
    x = rolling_apply(lambda x: adfuller(x)[0], Period, X, n_jobs=128, progress=True)
    x[:Period] = x[Period]
    return x

h = ADFSeries(T, period)
updateFigure(fig, ADF=h)

  0%|          | 0/4608 [00:00<?, ?it/s]

In [9]:
def hurstSeries(X: np.array, Period: int) -> np.array:
    x = rolling_apply(lambda x: hurst(x), Period, X, n_jobs=128, progress=True)
    x[:Period] = x[Period]
    return x

h = hurstSeries(T, period)
updateFigure(fig, hurstSeries=h)

  0%|          | 0/4608 [00:00<?, ?it/s]

In [10]:
h = HurstDSOD(T, period)
updateFigure(fig, HurstDSOD=h)

In [11]:
h = prun(lambda src, period: HurstRS(np.diff(src, prepend=0), period), T, period)
updateFigure(fig, HurstRS=h)

  0%|          | 0/128 [00:00<?, ?it/s]

In [12]:
h = prun(HurstDMA, T, period)
updateFigure(fig, HurstDMA=h)

  0%|          | 0/128 [00:00<?, ?it/s]